In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, classification_report,
    roc_curve
)

from xgboost import XGBClassifier
import matplotlib.pyplot as plt


In [46]:
import pandas as pd

df = pd.read_csv("exoplanets_clean_full.csv")
print("Loaded:", df.shape)


Loaded: (34993, 62)


In [47]:
engineered_cols = [
    "habitability_score",
    "stellar_compatibility_index",
    "earth_similarity_index",
    "orbital_stability_index",
    "star_planet_interaction_index",
    "distance_penalty"
]

[c for c in engineered_cols if c not in df.columns]


['habitability_score',
 'stellar_compatibility_index',
 'earth_similarity_index',
 'orbital_stability_index',
 'star_planet_interaction_index',
 'distance_penalty']

In [87]:
print(df.columns)
import numpy as np


# 1. Normalize required columns (safe min-max)


def safe_minmax(series):
    min_val = series.min()
    max_val = series.max()
    if max_val == min_val:
        return np.full(len(series), 0.5)
    return (series - min_val) / (max_val - min_val)

df["radius_norm"] = safe_minmax(df["pl_rade"])
df["temp_norm"]   = safe_minmax(df["pl_eqt"])
df["orbit_norm"]  = safe_minmax(df["pl_orbper"])

df["st_temp_norm"]   = safe_minmax(df["st_teff"])
df["st_radius_norm"] = safe_minmax(df["st_rad"])
df["st_lum_norm"]    = safe_minmax(df["st_lum"])


# 2. Habitability Score Index


df["habitability_score"] = (
    0.4 * (1 - df["radius_norm"]) +
    0.4 * (1 - abs(df["temp_norm"] - 0.5)) +
    0.2 * (1 - df["orbit_norm"])
)


# 3. Stellar Compatibility Index


df["stellar_compatibility_index"] = (
    0.4 * (1 - abs(df["st_temp_norm"] - 0.5)) +
    0.3 * (1 - df["st_radius_norm"]) +
    0.3 * (1 - df["st_lum_norm"])
)


# 4. Earth Similarity Index


df["earth_similarity_index"] = (
    (1 - abs(df["pl_rade"] - 1)) *
    (1 - abs(df["pl_eqt"] - 288) / 288)
)


# 5. Orbital Stability Index


df["orbital_stability_index"] = 1 / (1 + df["pl_orbper"])


# 6. Star–Planet Interaction Index


df["star_planet_interaction_index"] = (
    df["log_st_lum"] / (df["log_pl_orbsmax"] + 1)
)


# 7. Distance Penalty


df["distance_penalty"] = 1 / (1 + df["sy_dist"])

print("Engineered features created successfully")


Index(['pl_name', 'hostname', 'pl_rade', 'pl_bmasse', 'pl_eqt', 'pl_orbper',
       'pl_orbsmax', 'st_spectype', 'st_teff', 'st_rad', 'st_met', 'sy_dist',
       'st_lum', 'pl_density', 'pl_rade_missing', 'pl_bmasse_missing',
       'pl_density_missing', 'pl_eqt_missing', 'pl_orbper_missing',
       'pl_orbsmax_missing', 'st_teff_missing', 'st_rad_missing',
       'st_lum_missing', 'st_met_missing', 'sy_dist_missing',
       'st_spectype_missing', 'pl_rade_outlier', 'pl_bmasse_outlier',
       'pl_density_outlier', 'pl_eqt_outlier', 'pl_orbper_outlier',
       'pl_orbsmax_outlier', 'st_teff_outlier', 'st_rad_outlier',
       'st_lum_outlier', 'st_met_outlier', 'sy_dist_outlier', 'log_pl_rade',
       'log_pl_bmasse', 'log_st_lum', 'log_pl_orbsmax', 'stype_B', 'stype_F',
       'stype_G', 'stype_K', 'stype_M', 'stype_Other', 'stype_Unknown',
       'pl_rade_zs', 'pl_bmasse_zs', 'pl_density_zs', 'pl_eqt_zs',
       'pl_orbper_zs', 'pl_orbsmax_zs', 'st_teff_zs', 'st_lum_zs', 'st_met_zs',


In [51]:
engineered_cols = [
    "habitability_score",
    "stellar_compatibility_index",
    "earth_similarity_index",
    "orbital_stability_index",
    "star_planet_interaction_index",
    "distance_penalty"
]

[c for c in engineered_cols if c not in df.columns]


[]

In [ ]:

# Create Binary Habitability Label


df["habitable_label"] = (
    (df["pl_eqt"].between(200, 350)) &          # suitable temperature
    (df["pl_rade"].between(0.8, 2.0)) &         # Earth-like radius
    (df["stellar_compatibility_index"] > 0.5)  # suitable star
).astype(int)

print("Habitable label distribution:")
print(df["habitable_label"].value_counts())


Habitable label distribution:
habitable_label
0    34729
1      264
Name: count, dtype: int64


In [ ]:

# Feature Selection (No leakage)


FEATURE_COLUMNS = [
    "pl_rade",        # Planet radius
    "pl_bmasse",      # Planet mass
    "pl_eqt",         # Equilibrium temperature
    "pl_orbper",      # Orbital period
    "pl_orbsmax",     # Semi-major axis
    "pl_density",     # Planet density
    "st_teff",        # Stellar temperature
    "st_rad",         # Stellar radius
    "st_lum",         # Stellar luminosity
    "sy_dist"         # Distance from Earth
]

X = df[FEATURE_COLUMNS]
y = df["habitable_label"]

print("Feature matrix shape:", X.shape)
print("Target vector shape:", y.shape)


Feature matrix shape: (34993, 10)
Target vector shape: (34993,)


In [ ]:

#  Train–Test Split


from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Training samples:", X_train.shape[0])
print("Testing samples :", X_test.shape[0])


Training samples: 27994
Testing samples : 6999


In [ ]:

#  Train Random Forest Classifier


from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(
    n_estimators=300,
    max_depth=8,           # prevents overfitting
    min_samples_leaf=20,   # forces generalization
    class_weight="balanced",
    random_state=42
)

rf_model.fit(X_train, y_train)

print(" Random Forest model trained")


 Random Forest model trained


In [ ]:

# Predictions


rf_preds = rf_model.predict(X_test)
rf_probs = rf_model.predict_proba(X_test)[:, 1]


In [ ]:

#  Evaluation Metrics


from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score
)

print("🔹 Random Forest Evaluation")
print("Accuracy :", accuracy_score(y_test, rf_preds))
print("Precision:", precision_score(y_test, rf_preds))
print("Recall   :", recall_score(y_test, rf_preds))
print("F1-score :", f1_score(y_test, rf_preds))
print("ROC-AUC  :", roc_auc_score(y_test, rf_probs))


🔹 Random Forest Evaluation
Accuracy : 0.9991427346763824
Precision: 0.8983050847457628
Recall   : 1.0
F1-score : 0.9464285714285714
ROC-AUC  : 0.9998859123480868


In [ ]:

# Create Multi-Class Habitability Levels


import numpy as np

df["habitability_level"] = pd.cut(
    df["habitability_score"],
    bins=[-np.inf, 0.4, 0.7, np.inf],
    labels=[0, 1, 2]
)

print("Habitability level distribution:")
print(df["habitability_level"].value_counts())


Habitability level distribution:
habitability_level
2    34982
1       11
0        0
Name: count, dtype: int64


In [ ]:

#  Train–Test Split (Multi-class)


from sklearn.model_selection import train_test_split

Xm_train, Xm_test, ym_train, ym_test = train_test_split(
    X,
    df["habitability_level"],
    test_size=0.2,
    random_state=42,
    stratify=df["habitability_level"]
)


In [ ]:

#  Encode multi-class labels properly for XGBoost


from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

# Fit on FULL dataset 
df["habitability_level_enc"] = le.fit_transform(df["habitability_level"])

print("Encoded classes:", le.classes_)
print(df["habitability_level_enc"].value_counts())


Encoded classes: [1 2]
habitability_level_enc
1    34982
0       11
Name: count, dtype: int64


In [65]:
Xm_train, Xm_test, ym_train, ym_test = train_test_split(
    X,
    df["habitability_level_enc"],
    test_size=0.2,
    random_state=42,
    stratify=df["habitability_level_enc"]
)


In [67]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier(
    objective="multi:softprob",
    num_class=3,
    n_estimators=300,
    max_depth=5,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    eval_metric="mlogloss"
)

xgb_model.fit(Xm_train, ym_train)

print("XGBoost model trained successfully")


XGBoost model trained successfully


In [70]:
# Ensure true labels are integer type
ym_test = ym_test.astype(int)


In [74]:
print("xgb_preds shape:", xgb_preds.shape)
print("Unique predicted classes:", np.unique(xgb_preds))
df["habitability_level"] = pd.qcut(
    df["habitability_score"],
    q=3,
    labels=[0, 1, 2]
)


xgb_preds shape: (6999,)
Unique predicted classes: [0 1]


In [76]:
# Force class prediction (safe method)
xgb_preds = np.argmax(xgb_model.predict_proba(Xm_test), axis=1)
from sklearn.metrics import classification_report

print("🔹 XGBoost Multi-Class Evaluation")
print(classification_report(ym_test, xgb_preds))



🔹 XGBoost Multi-Class Evaluation
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00      6997

    accuracy                           1.00      6999
   macro avg       1.00      1.00      1.00      6999
weighted avg       1.00      1.00      1.00      6999



In [ ]:
from sklearn.calibration import CalibratedClassifierCV

# Probability Calibration using Platt Scaling (Sigmoid)


rf_calibrated = CalibratedClassifierCV(
    estimator=rf_model,   # already trained Random Forest
    method="sigmoid",     # Platt scaling (best for RF)
    cv=5                  # 5-fold internal calibration
)
rf_calibrated.fit(X_train, y_train)

print(" Probability calibration completed")


 Probability calibration completed


In [82]:
# Predict calibrated habitability probabilities
df["calibrated_habitability_probability"] = (
    rf_calibrated.predict_proba(X)[:, 1]
)
df_ranked_calibrated = (
    df.sort_values("calibrated_habitability_probability", ascending=False)
      .drop_duplicates(subset="pl_name")
)

df_ranked_calibrated[[
    "pl_name",
    "calibrated_habitability_probability"
]].head(10)


,pl_name,calibrated_habitability_probability
26723,Kepler-577 b,0.926170
26207,Kepler-54 d,0.926142
21083,Kepler-296 e,0.926080
19112,Kepler-235 e,0.926037
14209,Kepler-1652 b,0.926016
18772,Kepler-225 c,0.925972
28808,Kepler-737 b,0.925936
21078,Kepler-296 d,0.925853
16479,Kepler-186 e,0.925830
23936,Kepler-395 c,0.925816


In [83]:
from sklearn.metrics import accuracy_score, roc_auc_score

# Training predictions
train_preds = rf_model.predict(X_train)
train_probs = rf_model.predict_proba(X_train)[:, 1]

# Test predictions
test_preds = rf_model.predict(X_test)
test_probs = rf_model.predict_proba(X_test)[:, 1]


In [84]:
print("TRAIN Accuracy :", accuracy_score(y_train, train_preds))
print("TEST  Accuracy :", accuracy_score(y_test, test_preds))

print("TRAIN ROC-AUC  :", roc_auc_score(y_train, train_probs))
print("TEST  ROC-AUC  :", roc_auc_score(y_test, test_probs))


TRAIN Accuracy : 0.9990712295491891
TEST  Accuracy : 0.9991427346763824
TRAIN ROC-AUC  : 0.9999873767807481
TEST  ROC-AUC  : 0.9998859123480868


In [85]:
from sklearn.model_selection import cross_val_score

cv_scores = cross_val_score(
    rf_model,
    X,
    y,
    cv=5,
    scoring="roc_auc"
)

print("Cross-validation ROC-AUC scores:", cv_scores)
print("Mean CV ROC-AUC:", cv_scores.mean())


Cross-validation ROC-AUC scores: [1.         0.99942141 0.99980442 0.99998062 0.99992665]
Mean CV ROC-AUC: 0.999826620178806
